In [1]:
## PATH settings
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)
COMMON_PATH = os.path.join(project_root, 'common')

In [2]:
## Bespoke class imports
from common.database.adatabase import ADatabase
from common.processor.processor import Processor as p
from common.extractor.alpaca_extractor import AlpacaExtractor
from financial_common.indicator.indicator import Indicator
from financial_common.risk.benchmark import Benchmark
from financial_common.portfolio_management.kpi import KPI
from financial_common.metric.metric import Metric
from financial_common.portfolio_management.portfolio import Portfolio
import numpy as np
import matplotlib.pyplot as plt

In [3]:
## Import standard packages
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from time import sleep
import copy

In [4]:
orivault = ADatabase("ori")
alp = AlpacaExtractor(paper=False)

In [5]:
orivault.connect()
portfolio_dictionary = orivault.retrieve("results").to_dict("records")[0]
orivault.disconnect()
orivault.cloud_connect()
recommendations = orivault.retrieve("recommendations")
orivault.disconnect()

In [6]:
pm = Portfolio.from_dict(portfolio_dictionary)
recommendations = recommendations.to_dict("records")

In [7]:
recommendations

[{'year': 2025,
  'day': 143,
  'ticker': 'BNAIW',
  'date': Timestamp('2025-05-23 00:00:00'),
  'adjclose': 0.0102,
  'high': 0.02,
  'low': 0.0102,
  'volume': 665,
  'month': 5,
  'quarter': 2,
  'week': 21,
  'weekday': 4,
  'market_cap': 6.783,
  'standard_dev': 1,
  'risk': 1,
  'ema': 1.5369900074115241,
  'group_percentile': 1,
  'rank_percentile': 2,
  'major_key': '2025_143_1',
  'position_type': 1.0,
  'weight': 1.0}]

In [8]:

account = alp.account()
cash = round(float(account["cash"]),2)
for recommendation in recommendations:
    ticker = str(recommendation["ticker"])
    direction = int(recommendation["position_type"])
    ticker_data = alp.latest_bar(ticker)
    quote = alp.latest_quote(ticker)
    bid = float(quote["bp"])
    ask = float(quote["ap"])
    spread = round(float((ask-bid) * 0.1),4)
    adjclose = round(float(bid+spread),4)
    allocation = round(cash*recommendation["weight"],2)
    qty = int(allocation/adjclose)
    print("ticker:\t",ticker,"\nbid:\t ",bid,"\nask:\t", ask,"\nadjclose:\t",adjclose,"\nqty:\t",qty)
    # if direction == 1:
    #     print(alp.buy(ticker,adjclose,qty))
    # elif direction == -1:
    #     print(alp.sell(ticker,adjclose,qty))
    # else:
    #     print("invalid direction")


ticker:	 BNAIW 
bid:	  0.0103 
ask:	 0.0288 
adjclose:	 0.0122 
qty:	 9201


In [9]:
trade = alp.trades(ticker,alp.clock() - timedelta(days=3), alp.clock() - timedelta(days=1))
quotes = alp.quotes(ticker,alp.clock() - timedelta(days=3), alp.clock() - timedelta(days=1))

In [12]:
trade

,c,i,p,s,t,x,z
0,[@],1,0.0102,100,2025-05-22T13:43:49.193005746Z,Q,C
1,"[@, Q]",2,0.0102,100,2025-05-22T13:43:49.193018767Z,Q,C
2,[@],3,0.0102,100,2025-05-22T13:43:49.193484596Z,Q,C
3,[@],4,0.0102,100,2025-05-22T13:43:49.19349647Z,Q,C
4,"[@, I]",5,0.0102,26,2025-05-22T13:43:49.193497907Z,Q,C
5,[@],1,0.0103,100,2025-05-22T13:43:49.194805934Z,D,C
6,[@],2,0.0103,100,2025-05-22T13:43:49.195157339Z,D,C
7,"[@, I]",3,0.0103,26,2025-05-22T13:43:49.195505341Z,D,C
8,"[@, M]",1,0.0103,100,2025-05-22T20:00:00.055834169Z,P,C
9,"[@, M]",6,0.0102,26,2025-05-22T20:00:00.26402666Z,Q,C
